## Selección de caracteristicas
**Considera la siguiente pregunta**:  Dado un gran conjunto de datos (más de 1000 columnas con 10,000 filas (registros)), ¿cómo seleccionas las características útiles para construir un modelo supervisado?

De hecho, no hay una solución absoluta para esta pregunta, sino que está diseñada para probar tu capacidad de razonamiento lógico y habilidad de explicación.

Para complementar lo realizado en la clase de hoy, abordaremos una tarea diferente. Esta tarea tiene como objetivo predecir el nivel de riesgo (suscripción) del solicitante de seguro de vida. 

**Contexto**

Eres un científico de datos en una *start-up* con el potencial de tener un impacto muy grande en el negocio. Estás respaldado por una empresa con 140 años de experiencia en negocios.


*Prudential*, uno de los mayores emisores de seguros de vida en los EE. UU., está contratando científicos de datos apasionados para unirse a un grupo recién formado de Ciencia de Datos, resolviendo desafíos complejos e identificando oportunidades. Los resultados hasta ahora han sido impresionantes, pero queremos más.

**El reto**: En un mundo de compras con un clic y todo bajo demanda, el proceso de solicitud de seguro de vida es anticuado. Los clientes proporcionan información extensa para identificar la clasificación de riesgo y la elegibilidad, incluyendo la programación de exámenes médicos, un proceso que toma un promedio de 30 días.

*¿El resultado?* La gente se desanima. Es por eso que solo el 40% de los hogares en EE. UU. tiene un seguro de vida individual. Prudential quiere hacer que sea más rápido y menos laborioso para los nuevos y actuales clientes obtener una cotización, manteniendo los límites de privacidad.

Al desarrollar un modelo predictivo que clasifique con precisión el riesgo utilizando un enfoque más automatizado, puedes impactar enormemente la percepción pública de la industria.

Los resultados ayudarán a Prudential a comprender mejor el poder predictivo de los puntos de datos en la evaluación actual, permitiéndonos simplificar significativamente el proceso.

mayores detalles se pueden encontrar en la descripción del dataset: [prudential dataset](https://www.kaggle.com/c/prudential-life-insurance-assessment)



In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style='darkgrid')

from sklearn.feature_selection import RFECV, VarianceThreshold
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.preprocessing import OrdinalEncoder
random_state = 42

In [3]:
train_df = pd.read_csv('train.csv')
train_df.set_index('Id', inplace=True)
display(train_df.head())
X_train, y_train = train_df.iloc[:, :-1], train_df.iloc[:, -1]
print(X_train.shape)
print(y_train.shape)

,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,...,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
Id,,,,,,,,,,,,,,,,,,,,,
2,1,D3,10,0.076923,2,1,1,0.641791,0.581818,0.148536,...,0,0,0,0,0,0,0,0,0,8
5,1,A1,26,0.076923,2,3,1,0.059701,0.600000,0.131799,...,0,0,0,0,0,0,0,0,0,4
6,1,E1,26,0.076923,2,3,1,0.029851,0.745455,0.288703,...,0,0,0,0,0,0,0,0,0,8
7,1,D4,10,0.487179,2,3,1,0.164179,0.672727,0.205021,...,0,0,0,0,0,0,0,0,0,8
8,1,D2,26,0.230769,2,3,1,0.417910,0.654545,0.234310,...,0,0,0,0,0,0,0,0,0,8


(59381, 126)
(59381,)


In [7]:
y_train

Id
2        8
5        4
6        8
7        8
8        8
        ..
79142    4
79143    7
79144    8
79145    8
79146    7
Name: Response, Length: 59381, dtype: int64

y_train

## Limpieza de datos sencilla

Aquí realizamos una limpieza de datos sencilla. Por ejemplo, La mayoría de las implementaciones de modelos de aprendizaje automático no aceptan entradas de tipo cadena, por lo que debemos transformarlas en valores numéricos.

Actividad: analizar el siguiente proceso de limpieza de datos y documentar cada etapa


In [4]:
ordinal_y_mean_dict = {}

#
for col in X_train.select_dtypes(include='object').columns:
    ordinal_y_mean_dict[col]  = {index:i for i, index in enumerate(train_df.groupby(col)['Response'].mean().sort_values().index)}
    print(ordinal_y_mean_dict[col])
    X_train[col] = X_train[col].map(ordinal_y_mean_dict[col])
    
#
for col in X_train:
    if pd.isnull(X_train[col]).any():
        print('containing NA values:', col)

imputer = SimpleImputer(strategy='mean')
X_train2 = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
#
for col in X_train2:
    if pd.isnull(X_train2[col]).any():
        print('containing NA values:', col)

{'A7': 0, 'D1': 1, 'B1': 2, 'C2': 3, 'C1': 4, 'D2': 5, 'A8': 6, 'A2': 7, 'D3': 8, 'C3': 9, 'A3': 10, 'C4': 11, 'A1': 12, 'E1': 13, 'A4': 14, 'D4': 15, 'B2': 16, 'A5': 17, 'A6': 18}
containing NA values: Employment_Info_1
containing NA values: Employment_Info_4
containing NA values: Employment_Info_6
containing NA values: Insurance_History_5
containing NA values: Family_Hist_2
containing NA values: Family_Hist_3
containing NA values: Family_Hist_4
containing NA values: Family_Hist_5
containing NA values: Medical_History_1
containing NA values: Medical_History_10
containing NA values: Medical_History_15
containing NA values: Medical_History_24
containing NA values: Medical_History_32


In [5]:
## crear una copia
X_train2_unsup  = X_train2.copy() #deep copy


**Drop by the missing rate**

Eliminar columnas con una alta tasa de valores faltantes. El primer y más fácil enfoque es eliminar columnas en el enfoque no supervisado. Podemos eliminar columnas que tengan demasiados valores faltantes.
Consultar el uso de ``dropna``


**Drop by the variance**

Eliminar columnas con una varianza pequeña. Otra forma es eliminar columnas con una varianza demasiado pequeña, ya que estas columnas generalmente aportan poca información. Consultar el uso de ``VarianceThreshold``


**Supervised approach**

Un enfoque más sofisticado es hacerlo mediante aprendizaje supervisado. *Selección de características por modelo*. Algunos modelos de aprendizaje automático están diseñados para la selección de características, como la regresión lineal basada en L1 y los Árboles Extremadamente Aleatorizados ( Extra-trees model). En comparación con la regularización L2, la regularización L1 tiende a forzar los parámetros de las características no importantes a cero. (¿Sabes por qué?) Los árboles extremadamente aleatorizados dividen las hojas de manera aleatoria (no a través de la ganancia de información o la entropía). Las características importantes deberían seguir siendo más relevantes que las características no importantes (medidas por la importancia de características basada en la impureza).

In [6]:
X_train2_sup = X_train2.copy() #deep copy

X_model, X_valid, y_model, y_valid = train_test_split(X_train2_sup, y_train, stratify=y_train, random_state=random_state, test_size=.8)

model_dict = {'LogisticRegression': LogisticRegression(penalty='l1', solver='saga', C=2, multi_class='multinomial', n_jobs=-1, random_state=random_state)
             , 'ExtraTreesClassifier': ExtraTreesClassifier(n_estimators=200, max_depth=3, min_samples_leaf=.06, n_jobs=-1, random_state=random_state)
              , 'RandomForestClassifier': RandomForestClassifier(n_estimators=20, max_depth=2, min_samples_leaf=.1, random_state=random_state, n_jobs=-1)
             }




si el modelo es ``ExtraTreesClassifier`` o ``RandomForestClassifier``

importance_values = model.feature_importances_


Si el modelo es ``LogisticRegression``

importance_values = np.absolute(model.coef_) 

**Actividad**:  
 1. evaluar el modelo con todas las carácteristicas
 2. evaluar el modelo con las primeras 10 carácteristicas
 3. evaluar el modelo con las primeras 20 carácteristicas
 4. Observa cambios significativos en el rendimiento de los modelos?

**Eliminación Recursiva de Características (RFE)**. La segunda parte consiste en seleccionar la mejor combinación de características. Lo hacemos mediante la "Eliminación Recursiva de Características" (RFE). En lugar de construir un solo modelo, construimos n modelos (donde n = el número de características). 

